In [18]:
from pipeline import *
from model_generator import get_3_log_model



paths = get_all_paths("/Users/noah/Flatiron/NN/AudioRecognition/Pickles/")

data_list,labels_list = list_of_pickle_paths_to_data_list_and_labels(paths)
data,labels = concate_multiprocess_outputs(data_list,labels_list)


In [20]:
from multifileprocess import data_label_write
data_label_write(data,labels,"./saved_all_pkl")

In [28]:
print(data.keys())

dict_keys(['log_S', 'log_Sh', 'log_Sp', 'C', 'mfcc', 'delta_mfcc', 'delta2_mfcc'])


In [2]:
data["log_S"].shape

(1000, 128, 433)

In [3]:
res = data["log_S"].reshape(tuple(list(data["log_S"].shape)+[1]))

In [4]:
res[330][3][5]

array([-73.27483909])

In [5]:
data["log_S"][330][3][5]

-73.27483908553316

In [6]:
res = data["log_S"].reshape(55424000)

In [12]:
set([resi is None for resi in res])

{False}

In [25]:
model = get_3_log_model()

In [2]:
keys=["log_S","log_Sh","log_Sp"]

In [16]:
labels[0] = "10"
labels[1] = '101'
labels[2] = '11'
labels[3] = '12'
labels[4] = '15'


In [26]:

fit_dict(data,labels,model,keys,epochs=1)

Epoch 1/1
5200/5200 [==============================] - 9900s 2s/step - loss: 12.5294 - acc: 0.2196


In [27]:
model.save("./full_log3_model_save")

In [13]:
data["log_S"].shape

(1000, 128, 433)

In [8]:
input_vals = [reshape_for_model(data[key]) for key in keys]
labs = to_categorical(LabelEncoder().fit_transform(labels))

In [4]:
input_vals[0].shape

(1000, 128, 433, 1)

In [9]:
labs.shape

(1000, 1)

In [14]:
set([l[0] for l in labs])

{1.0, 10.0}

In [24]:
from keras import Model,Input
from keras.layers import Conv2D,Dense,Flatten,MaxPooling2D,concatenate
from keras.regularizers import l2
from keras.optimizers import Adam


def log_attach(input_layer):
    x = Conv2D(128,kernel_size=4,padding="valid",data_format="channels_last",activation="relu",use_bias=True)(input_layer)
    x = MaxPooling2D(padding="valid",pool_size=2)(x)
    x = Conv2D(64,kernel_size=4,padding="valid",data_format="channels_last",activation="relu",use_bias=True)(x)
    x = MaxPooling2D(padding="valid",pool_size=2)(x)
    x = Conv2D(5,kernel_size = 3,padding = "valid",data_format = "channels_last",activation = "relu",use_bias=True)(x)
    x = MaxPooling2D(padding="valid",pool_size=2)(x)
    x = Flatten()(x)
    outputs = Dense(100,activation="relu",use_bias=True)(x)
    return Model(inputs=input_layer,outputs=outputs)

def get_3_log_model():
    inputs = [Input((128,433,1)) for i in range(3)]
    mods = [log_attach(inputs[i]) for i in range(3)]

    com = concatenate([mod.output for mod in mods])
    x = Dense(100,activation="relu",use_bias=True)(com)
    x = Dense(50,activation="relu",use_bias=True)(x)
    x = Dense(5,activation="softmax",use_bias=True)(x)
    mod = Model(inputs = inputs,outputs = x)
    opt = Adam(lr=.01)
    mod.compile(optimizer=opt,loss='categorical_crossentropy',metrics=["accuracy"])
    return mod
